# Preprocessing of Crop Insurance Indemification Data
Crop insurance indeminfication is utilized to validate agro climatic zones. 

In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

**Description of Raw Data**
1. Summary Crop Insurance Cause of Loss was obtained from [USDA RMA] United States Department of Agriculture Risk Management Agency. https://www.rma.usda.gov/SummaryOfBusiness/CauseOfLoss
2. Meta data for crop insurance Cause of Loss is provided by RMA, https://www.rma.usda.gov/-/media/RMA/Cause-Of-Loss/Summary-of-Business-with-Month-of-Loss/colsommonth_allyears-pdf.ashx?la=en
3. Summary of Business for crop insurance, including total liabilities was obtained from [USDA RMA] United States Department of Agriculture Risk Management Agency. https://www.rma.usda.gov/SummaryOfBusiness

**Description of Processed Data**
1. County level 5-year total count of loss description per insurance event claim, and loss type as a percent of all loss type occurances
2. Crop specific county level 5-year mean total indemnification as a ratio of total liability. 

In [2]:
# County shape file to merge with county level crop insurance data. 
fp= 'C:/Users/sarahann.USERS/Desktop/code/us_county/US_County_Boundaries.shp' # county file
map_county = gpd.read_file(fp)
# Delete Non-contingous US counties 
ks_county = map_county.STATE.isin(['Kansas'])  
map_county = map_county[ks_county].loc[0:]
map_county['COUNTY']=map_county['COUNTY'].str.lower()

In [3]:
# Load most recent five years Cause of Loss Crop Insurance Indeminification Data
cs2021=pd.read_csv("C:/Users/sarahann.USERS/Desktop/code/crop_insurance_loss/2021.csv")
cs2020=pd.read_csv("C:/Users/sarahann.USERS/Desktop/code/crop_insurance_loss/2020.csv")
cs2019=pd.read_csv("C:/Users/sarahann.USERS/Desktop/code/crop_insurance_loss/2019.csv")
cs2018=pd.read_csv("C:/Users/sarahann.USERS/Desktop/code/crop_insurance_loss/2018.csv")
cs2017=pd.read_csv("C:/Users/sarahann.USERS/Desktop/code/crop_insurance_loss/2017.csv")

# Select Kansas from each year of indeminity
cs_2021 = cs2021[cs2021.STATECODE.isin([20])].loc[0:] 
cs_2020 = cs2020[cs2020.STATECODE.isin([20])].loc[0:] 
cs_2019 = cs2019[cs2019.STATECODE.isin([20])].loc[0:] 
cs_2018= cs2018[cs2018.STATECODE.isin([20])].loc[0:] 
cs_2017 = cs2017[cs2017.STATECODE.isin([20])].loc[0:] 

files = [cs2021, cs2020, cs2019, cs2018, cs2017]

file_list = []

# Process files to select for Kansas and remove cause of loss for irrigation infrastructure and price
for i in files:
    i = i[i.STATECODE.isin([20])].loc[0:] 
    i = i[~i.LOSSDESCRIPTION.isin(['Decline in Price', 'Failure of Irrigation Equipment','Failure of Irrigation Supply', 'Tornado', 'Inability to Prepare Land for Irrigation', 
                                   'Wildlife','Cyclone', 'Earthquake'])].loc[0:] 
    file_list.append(i)

**Loss Type. County level 5-year total count and percent representation of loss description per insurance event claim**

In [5]:
# Sum total indemnity for all observations for each county for each crop and each year
groupby= []

for i in file_list:
    x = i.groupby(by=['COUNTY','LOSSDESCRIPTION'])['LOSSDESCRIPTION'].size().to_frame().reindex()
    x['LOSS']=x['LOSSDESCRIPTION']
    x=x.drop(columns=['LOSSDESCRIPTION'])
    x = (pd.pivot_table(x,  index=['COUNTY'], columns=['LOSSDESCRIPTION'], aggfunc=np.sum)).droplevel(level=0, axis=1)
    x = x.fillna(0)
    groupby.append(x)

# Merge all files into a single file and sum    
loss_type = pd.concat([groupby[0], groupby[1], groupby[2], groupby[3], groupby[4]])
loss_type = pd.concat([groupby[0], groupby[1], groupby[2], groupby[3], groupby[4]])
loss_type=loss_type.reset_index()
loss_type=loss_type.groupby(by=['COUNTY']).sum()
loss_type = loss_type.fillna(0)

# Calculate percent for each loss type occurance as a ratio of all loss type occurances 
loss_type_per=loss_type.div(loss_type.sum(axis=1), axis=0)

# Final process of calculations and merge with a geopandas, spatial reference for county
loss_type=loss_type.reset_index()
loss_type.COUNTY=loss_type.COUNTY.str.strip().str.lower()
loss_type=map_county.merge(loss_type, on='COUNTY')

loss_type_per=loss_type_per.reset_index()
loss_type_per.COUNTY=loss_type_per.COUNTY.str.strip().str.lower()
loss_type_per=map_county.merge(loss_type_per, on='COUNTY')

# Export to CSV and Shape files
loss_type.to_csv('loss_type_5yr_count.csv')
loss_type.to_file('loss_type_5yr_count.shape')
loss_type_per.to_csv('loss_type_percent.csv')
loss_type_per.to_file('loss_type_per.shape')

In [6]:
loss_type_per.columns.unique()

Index(['STFIPS', 'CTFIPS', 'STATE', 'COUNTY', 'VERSION', 'Shape_Leng',
       'Shape_Area', 'geometry', 'ARPI/SCO/ECO/STAX/MP/HIP WI Crops Only',
       'All Other Causes', 'Asian Soybean Rust', 'Cold Wet Weather',
       'Cold Winter', 'Drought', 'Excess Moisture/Precipitation/Rain', 'Fire',
       'Flood', 'Freeze', 'Frost', 'Hail', 'Heat', 'Hot Wind', 'Insects',
       'Mycotoxin', 'Other (Snow, Lightning, Etc.)', 'Plant Disease',
       'Wind/Excess Wind', 'ARPI/SCO/STAX/MP/HIP WI Crops Only',
       'ARPI/SCO/STAX/MP Crops Only', 'Falling Numbers'],
      dtype='object')

**Ratio of Indemnity to Liability Premium. County Level 5-year mean by crop**

In [7]:
# Load Summary of Business Data 
df = pd.read_csv("C:/Users/sarahann.USERS/Desktop/code/ks_agro_climate/crop_insurance_summary_of_business.csv")
print(df.columns.unique())
df.head(3)

Index(['Commodity Year', 'State Abbrv', 'County Name', 'Commodity Name',
       'Delivery Type Code', 'Delivery Type Name', 'Insurance Plan Abbrv',
       'Cov. Level Percent', 'Policies Sold', 'Policies Earning Prem',
       'Policies Indemnified', 'Units Earning Prem', 'Units Indemnified',
       'Quantity', 'Quantity Type', 'Companion/Endorsed Acres',
       'Liabilities ($)', 'Total Prem ($)', 'Subsidy ($)', 'Indemnity ($)',
       'EFA Prem Discount ($)', 'Addnl Subsidy ($)', 'State Subsidy ($)',
       'Earn Prem Rate', 'Loss Ratio'],
      dtype='object')


,Commodity Year,State Abbrv,County Name,Commodity Name,Delivery Type Code,Delivery Type Name,Insurance Plan Abbrv,Cov. Level Percent,Policies Sold,Policies Earning Prem,...,Companion/Endorsed Acres,Liabilities ($),Total Prem ($),Subsidy ($),Indemnity ($),EFA Prem Discount ($),Addnl Subsidy ($),State Subsidy ($),Earn Prem Rate,Loss Ratio
0,2017,KS,All Other Counties,All Other Commodities,RBUP,Reinsured Buyup,DO,0.75,1,1,...,0,614529,17838,9811,0,0,0,0,0.029027,0.0
1,2017,KS,All Other Counties,All Other Commodities,RCAT,Reinsured Catastrophic,DO,0.50,5,4,...,0,5099384,28101,28101,0,0,0,0,0.005510,0.0
2,2017,KS,Allen,Corn,RBUP,Reinsured Buyup,RP,0.50,1,0,...,0,0,0,0,0,0,0,0,0.000000,0.0


In [8]:
# Groupby County and Commodity for Liabilities
net_liab = df.groupby(by=['County Name','Commodity Name'])['Liabilities ($)'].sum().to_frame()
net_liab=(pd.pivot_table(net_liab,  index=['County Name'], columns=['Commodity Name'], aggfunc=np.sum)).droplevel(level=0, axis=1)
print(net_liab.columns.unique())
net_liab.head(3)

Index(['All Other Commodities', 'Annual Forage', 'Barley', 'Canola', 'Corn',
       'Cotton', 'Dry Beans', 'Dry Peas', 'Grain Sorghum', 'Hemp',
       'Hybrid Sorghum Seed', 'Millet', 'Oats', 'Onions',
       'Pasture,Rangeland,Forage', 'Potatoes', 'Rye', 'Sesame',
       'Silage Sorghum', 'Soybeans', 'Sunflowers', 'Triticale', 'Wheat'],
      dtype='object', name='Commodity Name')


Commodity Name,All Other Commodities,Annual Forage,Barley,Canola,Corn,Cotton,Dry Beans,Dry Peas,Grain Sorghum,Hemp,...,Onions,"Pasture,Rangeland,Forage",Potatoes,Rye,Sesame,Silage Sorghum,Soybeans,Sunflowers,Triticale,Wheat
County Name,,,,,,,,,,,,,,,,,,,,,
All Other Counties,37786949.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Allen,2089.0,NaN,NaN,NaN,52239364.0,NaN,NaN,NaN,218504.0,NaN,...,NaN,2491419.0,NaN,NaN,NaN,NaN,78546754.0,NaN,NaN,5121920.0
Anderson,869885.0,0.0,NaN,NaN,79924076.0,NaN,NaN,NaN,789563.0,NaN,...,NaN,7647811.0,NaN,NaN,NaN,NaN,105896466.0,NaN,NaN,3216666.0


In [9]:
# Groupby County and Commodity for Indemnities
net_ind = df.groupby(by=['County Name','Commodity Name'])['Indemnity ($)'].sum().to_frame()
net_ind = (pd.pivot_table(net_ind,  index=['County Name'], columns=['Commodity Name'], aggfunc=np.sum)).droplevel(level=0, axis=1)
print(net_ind.columns.unique())
net_ind.head(3)

Index(['All Other Commodities', 'Annual Forage', 'Barley', 'Canola', 'Corn',
       'Cotton', 'Dry Beans', 'Dry Peas', 'Grain Sorghum', 'Hemp',
       'Hybrid Sorghum Seed', 'Millet', 'Oats', 'Onions',
       'Pasture,Rangeland,Forage', 'Potatoes', 'Rye', 'Sesame',
       'Silage Sorghum', 'Soybeans', 'Sunflowers', 'Triticale', 'Wheat'],
      dtype='object', name='Commodity Name')


Commodity Name,All Other Commodities,Annual Forage,Barley,Canola,Corn,Cotton,Dry Beans,Dry Peas,Grain Sorghum,Hemp,...,Onions,"Pasture,Rangeland,Forage",Potatoes,Rye,Sesame,Silage Sorghum,Soybeans,Sunflowers,Triticale,Wheat
County Name,,,,,,,,,,,,,,,,,,,,,
All Other Counties,19967.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Allen,0.0,NaN,NaN,NaN,3560689.0,NaN,NaN,NaN,20578.0,NaN,...,NaN,252203.0,NaN,NaN,NaN,NaN,3477325.0,NaN,NaN,640854.0
Anderson,193354.0,0.0,NaN,NaN,2931952.0,NaN,NaN,NaN,86930.0,NaN,...,NaN,527722.0,NaN,NaN,NaN,NaN,2740099.0,NaN,NaN,263277.0


In [10]:
# Divide Indeminity by Liability for Ratio
liab_ratio_by_crop=net_ind/net_liab
liab_ratio_by_crop.drop(['All Other Commodities'], axis=1, inplace=True)
liab_ratio_by_crop.drop(['All Other Counties'], axis=0, inplace=True)

# Merge County-Crop liability to indemnity ratios with geopandas for county spatial data
liab_ratio_by_crop=liab_ratio_by_crop.reset_index()
liab_ratio_by_crop['County Name']=liab_ratio_by_crop['County Name'].str.strip().str.lower()
liab_ratio_by_crop.rename(columns=({'County Name':'COUNTY'}), inplace=True)
liab_ratio_by_crop=map_county.merge(liab_ratio_by_crop, on='COUNTY')

#Export as a CSV and Shape File
liab_ratio_by_crop.to_csv('liab_ratio_by_crop.csv')
liab_ratio_by_crop.to_file('liab_ratio_by_crop.shape')
liab_ratio_by_crop.head(3)

,STFIPS,CTFIPS,STATE,COUNTY,VERSION,Shape_Leng,Shape_Area,geometry,Annual Forage,Barley,...,Onions,"Pasture,Rangeland,Forage",Potatoes,Rye,Sesame,Silage Sorghum,Soybeans,Sunflowers,Triticale,Wheat
0,20,20023,Kansas,cheyenne,05,2.145800,0.277993,"POLYGON ((-102.04981 39.57406, -102.04993 39.5...",0.331577,0.117000,...,NaN,0.236326,NaN,NaN,NaN,NaN,0.032165,0.077457,NaN,0.059602
1,20,20153,Kansas,rawlins,05,2.213924,0.291303,"POLYGON ((-101.41391 39.56832, -101.41361 39.5...",0.283451,0.358995,...,NaN,0.236017,NaN,NaN,NaN,NaN,0.088354,0.050865,NaN,0.052791
2,20,20089,Kansas,jewell,05,2.016403,0.249024,"POLYGON ((-98.50527 39.56760, -98.50530 39.567...",NaN,0.038528,...,NaN,0.152374,NaN,NaN,NaN,NaN,0.026751,0.095459,NaN,0.052105


In [11]:
liab_ratio_by_crop

,STFIPS,CTFIPS,STATE,COUNTY,VERSION,Shape_Leng,Shape_Area,geometry,Annual Forage,Barley,...,Onions,"Pasture,Rangeland,Forage",Potatoes,Rye,Sesame,Silage Sorghum,Soybeans,Sunflowers,Triticale,Wheat
0,20,20023,Kansas,cheyenne,05,2.145800,0.277993,"POLYGON ((-102.04981 39.57406, -102.04993 39.5...",0.331577,0.117000,...,NaN,0.236326,NaN,NaN,NaN,NaN,0.032165,0.077457,NaN,0.059602
1,20,20153,Kansas,rawlins,05,2.213924,0.291303,"POLYGON ((-101.41391 39.56832, -101.41361 39.5...",0.283451,0.358995,...,NaN,0.236017,NaN,NaN,NaN,NaN,0.088354,0.050865,NaN,0.052791
2,20,20089,Kansas,jewell,05,2.016403,0.249024,"POLYGON ((-98.50527 39.56760, -98.50530 39.567...",NaN,0.038528,...,NaN,0.152374,NaN,NaN,NaN,NaN,0.026751,0.095459,NaN,0.052105
3,20,20183,Kansas,smith,05,1.992673,0.244139,"POLYGON ((-99.04409 39.56786, -99.04429 39.567...",NaN,NaN,...,NaN,0.117586,NaN,NaN,NaN,0.017509,0.025075,0.000000,NaN,0.043569
4,20,20147,Kansas,phillips,05,1.990133,0.243669,"POLYGON ((-99.62795 39.56744, -99.62777 39.573...",NaN,NaN,...,NaN,0.139701,NaN,NaN,NaN,0.000000,0.035938,0.035199,NaN,0.028248
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100,20,20077,Kansas,harper,05,1.864473,0.211051,"POLYGON ((-98.34719 36.99796, -98.34719 36.998...",0.005486,0.241634,...,NaN,0.098047,NaN,NaN,0.033553,NaN,0.093691,0.000000,NaN,0.080160
101,20,20099,Kansas,labette,05,1.664944,0.171715,"POLYGON ((-95.52121 37.38398, -95.51935 37.383...",NaN,NaN,...,NaN,0.086074,NaN,NaN,NaN,NaN,0.074855,NaN,NaN,0.316113
102,20,20033,Kansas,comanche,05,1.853777,0.207567,"POLYGON ((-99.45620 36.99947, -99.45658 36.999...",0.052129,NaN,...,NaN,0.120811,NaN,NaN,NaN,NaN,0.035385,NaN,NaN,0.092504
103,20,20021,Kansas,cherokee,05,1.594180,0.155286,"POLYGON ((-95.00762 36.99951, -95.01143 36.999...",NaN,NaN,...,NaN,0.095931,NaN,NaN,NaN,NaN,0.041704,NaN,NaN,0.148284
